# Administration: Manage inactive users

> * ✏️ Needs Configuration
* 🔒 Requires Administrator Privileges
* 🗄️ Administration
* 👤 User Management
* 🔔 Notifications

Over time, an Organization can become cluttered with many users, some of whom haven't logged in for several months. This notebook will search through all `Users` in an Organization and disable those users that haven't logged in for a certain amount of days. This notebook will also email users a warning that they haven't logged in for a certain amount of days. This notebook can be a powerful tool to automatically manage these inactive users.

To get started, import the necessary libraries and connect to our GIS:

In [36]:
import csv
import smtplib
import time
from datetime import timedelta
import datetime as dt
import logging
log = logging.getLogger()

import pandas
from IPython.display import display, HTML

from arcgis.gis import GIS

gis = GIS("home")

Next, we will create our function that sends out notifications.

## Notifications

An important part of this process is notifying the user about their inactivity. We will achieve this by connecting to an external SMTP server and sending out emails, but you can write _any_ code that connects to _any_ external service to send out notifications. __View the 'Notifications' notebook in the examples gallery for more information__.

__<span style="color:red;">You MUST modify the below cell</span>__ to update `secret_csv_item_id`, `smtp_server_url`, `username`, and any other information needed to connect to your external smtp server. This includes making sure your _secrets.csv_ file item contains the `smtp_email_password` entry.

In [37]:
# Helper function for using your private secrets.csv file
def get_secrets(gis=gis,
                secret_csv_item_id = '<YOUR_ITEM_ID_HERE>'):
    """Returns a dict of { secret_key : secret_value } from the 
    secrets.csv item. See the 'Notifications' notebook in the 
    examples gallery for more information.
    """
    try:
        item = gis.content.get(secret_csv_item_id)
        with open(item.download(), 'r') as local_item_file:
            reader = csv.DictReader(local_item_file)
            return { rows['secret_key'] : rows['secret_value'] \
                     for rows in reader }
    except Exception:
        return {}

SECRETS = get_secrets()

# Helper function to send out emails through an SMTP server
def send_email_smtp(recipients, message,
                    subject="Message from your Notebook"):
    """Sends the `message` string to all of the emails in the 
    `recipients` list using the configured SMTP email server. 
    See the 'Notifications' notebook in the examples gallery
    for more information.
    """
    try:
        # Set up server and credential variables
        smtp_server_url = "smtp.example.com"
        smtp_server_port = 587
        sender = "your_sender@example.com"
        username = "your_username"
        password = SECRETS["smtp_email_password"]

        # Instantiate our server, configure the necessary security
        server = smtplib.SMTP(smtp_server_url, smtp_server_port)
        server.ehlo()
        server.starttls() # Needed if TLS is required w/ SMTP server
        server.login(username, password)
    except Exception as e:
        log.warning("Error setting up SMTP server, couldn't send " +
                    f"message to {recipients}")
        raise e

    # For each recipient, construct the message and attempt to send
    did_succeed = True
    for recipient in recipients:
        try:
            message_body = '\r\n'.join(['To: {}'.format(recipient),
                                        'From: {}'.format(sender),
                                        'Subject: {}'.format(subject),
                                        '{}'.format(message)])
            message_body = message_body.encode("utf-8")
            server.sendmail(sender, [recipient], message_body)
            print(f"SMTP server returned success for sending email "\
                  f"to {recipient}")
        except Exception as e:
            log.warning(f"Failed sending message to {recipient}")
            log.warning(e)
            did_succeed = False
    
    # Cleanup and return
    server.quit()
    return did_succeed

Item does not exist or is inaccessible.


Make sure to independently test that your notifications function works before proceeding forward with this notebook.

## Configure Behavior

Now that we have a function that sends out notifications, let's configure some variables specific to our organization that will tell our notebook how we want it to run. First, let's specify our time windows for disabling users. The default behavior will email users warning them that they have not logged in for 60 days (2 months), and then notify them that their account might be deleted once they have not logged in for 90 days (3 months).

Modify the below cell to change this default behavior.

In [38]:
# The number of days a user is inactive for before...
NUM_INACTIVE_DAYS_TO_NOTIFY = 365 # we notify about their inactivity
NUM_INACTIVE_DAYS_TO_DISABLE = 700 # we delete their account

Next, we will specify if we want to email a summary email of the users notified/disabled during this notebook run. The default behavior is to send this report to whomever is specified in the `NOTEBOOK_REPORT_RECIPIENTS` list of strings. Add email addresses to that list, like this:

```python
NOTEBOOK_REPORT_RECIPIENTS = ['somebody@example.com',
                              'somebody_else@example.com']
```

In [39]:
# Email these people a report about who this notebook notified/disabled
EMAIL_NOTEBOOK_REPORT = True
NOTEBOOK_REPORT_RECIPIENTS = []

Finally, this notebook has a `USER_DISABLE_NOTIFY_PROTECTION` flag that by default is set to `True`. This safety flag will prevent this notebook from physically calling `user.disable()` or sending out any emails. This notebook is powerful and potentially dangerous, since it can disable many user accounts and send out emails to _everyone_ in an organization. Proceed with caution. 

__<span style="color:red;">Only modify the below cell if you understand the implications</span>__.

In [40]:
USER_DISABLE_NOTIFY_PROTECTION = True

## Datetimes

A core component of this notebook will be testing how many days since a user logged in. The below code cell creates a helper function that can calculate this value using the `user.lastLogin` property and the builtin `datetime` library.

In [41]:
def datetime_of_last_login(user) -> dt.datetime:
    """Returns a datetime instance of when the user last logged in.
    
    The `user.lastLogin` property returns the milliseconds since the 
    epoch, whereas a standard timestamp is the seconds since the epoch. 
    Therefore, we must divide by 1000.
    """
    timestamp_of_last_login = int(user.lastLogin / 1000)
    last_login = dt.datetime.fromtimestamp(timestamp_of_last_login)
    return last_login

This helper function returns the number of days between the last login and today and an `int`.

In [42]:
def num_days_since_last_login(user) -> int:
    """Returns the number of days since a user last logged in. 

    Subtracting the `last_login` and `now` datetime instances will
    yield a timedelta instance. This timedelta instance has a 
    `.days` property, which is what we want to return
    """
    last_login = datetime_of_last_login(user)
    now = dt.datetime.now()
    return (now - last_login).days

This helper function calculates the planned datetime for deleting a user.

In [43]:
def datetime_of_account_deletion(user) -> dt.datetime:
    """Returns the datetime of when the account is planned to be deleted
    
    This is calculated based off of NUM_INACTIVE_DAYS_TO_DELETE variable,
    by adding a timedelta to a datetime (returns a datetime)
    """
    last_login = datetime_of_last_login(user)
    return last_login + timedelta(days=NUM_INACTIVE_DAYS_TO_DISABLE)

## Notify/Disable

Now, let's create a function that will notify a user when their account is scheduled to be disabled.

In [44]:
def notify_user(user):
    """Called when in the 'notify' range. Will send an email to the user,
    notifying them that their account will be disabled.
    """
    num_days_last_login = num_days_since_last_login(user)
    num_days_delete = NUM_INACTIVE_DAYS_TO_DISABLE - num_days_last_login
    date_of_deletion = datetime_of_account_deletion(user).date()
    if num_days_delete > 0:
        delete_warning = "could potentially be deleted in "\
            f"{num_days_delete} days on {date_of_deletion}"
    else:
        delete_warning = "could potentially be deleted"
    message = f"Hello {user.username},\n"\
              f"\n"\
              f"Your account at {gis.url} is hasn't been logged in "\
              f"for {num_days_last_login} days, and {delete_warning}. "\
              f"Please login to your account to prevent deletion, "\
              f"or contact your org admin for more information.\n"\
              f"\n"\
              f"Account URL: {user.homepage}"
    send_email_smtp([user.email], message,
                    subject="Your Account Could Be Disabled")

Then, let's create a function that will disable a user and email them a notification.

In [45]:
def disable_user(user):
    """Called when in the 'disable' range. Will disable the user, and 
    will send them an email that their account is disabled.
    """
    if not user.disabled:
        print(f"Disabling user {user.username}")
        user.disable()
        message = f"Hello {user.username},\n"\
              f"\n"\
              f"Your account at {gis.url} has been disabled since you "\
              f"haven't logged in in {NUM_INACTIVE_DAYS_TO_DISABLE} "\
              f"days.\n\nContact your org admin to regain access."
        send_email_smtp([user.email], message,
                        subject="Your Account is Disabled")

## Miscellanous Functionality

Let's create a function that writes a .csv file of all users, and if they are in the notify or disable range of inactivity. 

In [46]:
workspace = "/arcgis/home/"

def write_csv(users_notify_range, users_disable_range):
    """Given two lists of users to notify/disable, write a CSV
    with 1 row per user, stating if in the notify/disable range """
    timestamp = dt.datetime.now().isoformat()
    file_path = f'{workspace}/INACTIVE_USERS--{timestamp}.csv'
    print (file_path)
    with open(file_path, 'w') as file:
        writer = csv.DictWriter(file, ['user', 'in_notify_range',
                                       'in_disable_range'])
        writer.writeheader()
        for user in gis.users.search(max_users =600):
            writer.writerow({
                'user' : user.username,
                'in_notify_range' : bool(user in users_notify_range),
                'in_disable_range' : bool(user in users_disable_range)})
    return gis.content.add({}, file_path)

Then, let's create a function that emails specific individuals a notebook report of notified/disabled users.

In [47]:
def stringify_users(users):
    """Takes a list of users, and returns a string representation"""
    if users:
        return ", ".join(user.username for user in users)
    else:
        return "No Users"

def email_notebook_report_summary(notified_users, disabled_users,
                                  csv_item):
    """After the notebook runs, send out a summary report email to
    the specified people about who was notified/disabled
    """
    message = f"'Manage Inactive Users' notebook finished running.\n"\
              f"-----\n"\
              f"CSV output: {csv_item.homepage}\n"\
              f"-----\n"\
              f"Users notified: {stringify_users(notified_users)}\n"\
              f"-----\n"\
              f"Disabled users: {stringify_users(disabled_users)}"
    send_email_smtp(NOTEBOOK_REPORT_RECIPIENTS, message,
                    subject="'Manage Inactive Users' notebook run")

Finally, let's create a simple function that tests to make sure this notebook is running against a GIS object with the correct admin credentials.

In [48]:
def precondition_check():
    """Checks if running as admin by checking 'lastLogin' prop of User"""
    if 'lastLogin' not in gis.users.me:
        raise Exception("You don't have the proper permissions to "\
                        "run this notebook; you must be an admin.")

## main()

Finally, let's create our `main()` function that links together all our previously defined functions that search through users, notify/disable them if they haven't logged in for the specified number of days, and display the proper notebook outputs.

In [49]:
users_to_notify = []
users_to_disable = []
csv_item = None

def main():
    # Tell user we're running, initialize
    print("Notebook is now running, please wait...\n-----")
    global users_to_notify, users_to_disable, csv_item
    precondition_check()
    users_to_notify = []
    users_to_disable = []

    # Stage an in-memory list of all users to notify/delete
    for user in gis.users.search(max_users=600):
        print(user)
        num_days = num_days_since_last_login(user)
        if (num_days > NUM_INACTIVE_DAYS_TO_NOTIFY) and \
           (num_days < NUM_INACTIVE_DAYS_TO_DISABLE):
            # If in the 'notify' timeframe, but not long enough to delete
            users_to_notify.append(user)
        elif (num_days >= NUM_INACTIVE_DAYS_TO_DISABLE):
            # If in the 'delete' timeframe
            users_to_disable.append(user)

    # Write the in-memory lists to a CSV
    csv_item = write_csv(users_to_notify, users_to_disable)
    display(HTML("<h3>CSV of all users in notify/disable range</h3>"))
    display(csv_item)

    # Check if we should notify/delete the users in the in-memory lists
    if USER_DISABLE_NOTIFY_PROTECTION:
        log.warning(f"Not disabling/notifying any users, since "\
                    f"USER_DISABLE_NOTIFY_PROTECTION config variable "\
                    f"is set to `True`: Set to `False` to have "\
                    f"this notebook actually notify/disable users.")
    else:
        # If configured, notify/disable the users in the in-memory lists
        print(f"Notifying {stringify_users(users_to_notify)}")
        print(f"Disabling {stringify_users(users_to_disable)}")
        for user in users_to_notify:
            notify_user(user)
        for user in users_to_disable:
            disable_user(user)

        # If configured, email a report of notified/deleted users
        if EMAIL_NOTEBOOK_REPORT:
            email_notebook_report_summary(users_to_notify,
                                          users_to_disable,
                                          csv_item)

    # Tell user we're finished
    print("-----\nNotebook completed running.")

We have just defined a `main()` function, but we haven't called it yet. If you've modified the notebook, follow these steps:
1. __Double check the notebook content__. Make sure no secrets are visible in the notebook, delete unused code, refactor, etc.
2. Save the notebook
3. In the 'Kernel' menu, press 'Restart and Run All' to run the whole notebook from top to bottom

Now, `main()` can be called.

In [50]:
main()

Notebook is now running, please wait...
-----
<User username:adam.scott_RSPB>
<User username:adrian.hughes_Ext>
<User username:adrian.hughes_RSPB>
<User username:adrian_hughes_gough_RSPB>
<User username:ahmedendris2_EXT>
<User username:Ajhermae.White_EXT>
<User username:Alasdair.Fraser_RSPB>
<User username:alastair.moralee_PL>
<User username:alazar.ruffo_EXT>
<User username:alexander.falkingham_RSPB>
<User username:Alexandr.Bolbocean_Ext>
<User username:Alexandru.Mormeci_Ext>
<User username:ali.plummer_RSPB>
<User username:alice.edwards_RSPB>
<User username:alice.skehel_RSPB>
<User username:alina.costea_RSPB>
<User username:alison.bennett_RSPB>
<User username:alison.beresford_RSPB>
<User username:alison_phillip_rspb>
<User username:alistair.smith_RSPB>
<User username:Allan.Perkins_RSPB>
<User username:Allan.Robertson_RSPB>
<User username:AlysPerry_Ext>
<User username:Amity.AllenBills_RSPB>
<User username:amy.morrison_RSPB>
<User username:amy.vanstone_RSPB>
<User username:andrew.dodd_RS

<Item title:"INACTIVE_USERS--2020-04-13T09:42:34.132236" type:CSV owner:adrian.hughes_RSPB>

Not disabling/notifying any users, since USER_DISABLE_NOTIFY_PROTECTION config variable is set to `True`: Set to `False` to have this notebook actually notify/disable users.


-----
Notebook completed running.


Congratulations! If all is setup correctly, emails should be sent out for users in the 'notify' range, users in the 'disable' range should be disabled, and the notebook should output the correct artifacts.

## Post Processing

The `INACTIVE_USERS.csv` file/item contains a list of all users, if they are in the `notify_range`, if they are in the `in_disable_range`, or if they are in neither range. This file can be viewed and analyzed using the `pandas` package. This code cell will convert any `.csv` Item to a pandas `DataFrame`; we will be converting the `INACTIVE_USERS.csv` file.

In [51]:
def csv_item_to_dataframe(item):
    """Takes in an Item instance of a `.csv` file,
    returns a pandas DataFrame
    """
    downloaded_csv_file_path = item.download()
    return pandas.read_csv(downloaded_csv_file_path)

df = csv_item_to_dataframe(csv_item)
df.head()

,user,in_notify_range,in_disable_range
0,adam.scott_RSPB,False,True
1,adrian.hughes_Ext,False,False
2,adrian.hughes_RSPB,False,False
3,adrian_hughes_gough_RSPB,False,False
4,ahmedendris2_EXT,True,False


# Conclusion

This notebook provided you the workflow for searching through all Users in an Organization, and will notify/disable them if they haven't logged in for a certain amount of days. This notebook can be a powerful tool to automatically manage these inactive users since Organizations can become cluttered with many users over time.

### Related Notebooks
For related notebooks, search for the following in your samples notebook gallery:

- Notifications
- Validate User Profiles